# Análisis de Anomalías en Sistema Solar Fotovoltaico

Este notebook implementa tres métodos de detección de anomalías usando datos de:
- Datos ambientales (temperatura, velocidad y dirección del viento)
- Datos de irradiancia solar
- Datos eléctricos de 24 inversores

## 1. Importación de librerías y carga de datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Cargar los datos
environment_data = pd.read_csv('environment_data.csv')
irradiance_data = pd.read_csv('irradiance_data.csv')
electrical_data = pd.read_csv('electrical_data.csv')

# Convertir columnas de fecha a datetime
environment_data['measured_on'] = pd.to_datetime(environment_data['measured_on'])
irradiance_data['measured_on'] = pd.to_datetime(irradiance_data['measured_on'])
electrical_data['measured_on'] = pd.to_datetime(electrical_data['measured_on'])

print(f"Datos ambientales: {environment_data.shape}")
print(f"Datos de irradiancia: {irradiance_data.shape}")
print(f"Datos eléctricos: {electrical_data.shape}")

## 2. Análisis exploratorio de datos

In [ ]:
# Visualizar información básica de cada dataset
print("=== INFORMACIÓN DE DATOS AMBIENTALES ===")
print(environment_data.info())
print("\n=== ESTADÍSTICAS DESCRIPTIVAS ===")
print(environment_data.describe())

In [ ]:
# Verificar valores nulos
print("Valores nulos en environment_data:", environment_data.isnull().sum().sum())
print("Valores nulos en irradiance_data:", irradiance_data.isnull().sum().sum())
print("Valores nulos en electrical_data:", electrical_data.isnull().sum().sum())

## 3. Unión y transformación de datos

Vamos a unir las tablas usando la columna `measured_on` como clave

In [ ]:
# Unir environment e irradiance primero
merged_data = pd.merge(environment_data, irradiance_data, on='measured_on', how='inner')
print(f"Después de unir environment e irradiance: {merged_data.shape}")

# Luego unir con electrical
final_data = pd.merge(merged_data, electrical_data, on='measured_on', how='inner')
print(f"Datos finales combinados: {final_data.shape}")

In [ ]:
# Crear características adicionales
# Calcular la potencia total AC de todos los inversores
ac_power_cols = [col for col in final_data.columns if 'ac_power' in col]
final_data['total_ac_power'] = final_data[ac_power_cols].sum(axis=1)

# Calcular eficiencia promedio (si tenemos datos DC)
dc_power_cols = [col for col in final_data.columns if 'dc_current' in col and 'dc_voltage' in col]

# Agregar características temporales
final_data['hour'] = final_data['measured_on'].dt.hour
final_data['day_of_week'] = final_data['measured_on'].dt.dayofweek
final_data['month'] = final_data['measured_on'].dt.month

print("Características creadas exitosamente")

## 4. Implementación de detectores de anomalías

### 4.1 Método 1: Isolation Forest

In [ ]:
# Seleccionar características para el modelo
feature_cols = ['ambient_temperature_o_149575', 'wind_speed_o_149576', 
                'poa_irradiance_o_149574', 'total_ac_power', 'hour']

# Preparar datos
X = final_data[feature_cols].copy()
X = X.fillna(X.mean())  # Imputar valores faltantes

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Implementar Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42, n_estimators=100)
final_data['anomaly_isolation'] = iso_forest.fit_predict(X_scaled)
final_data['anomaly_score_isolation'] = iso_forest.score_samples(X_scaled)

# Contar anomalías
n_anomalies_iso = (final_data['anomaly_isolation'] == -1).sum()
print(f"Anomalías detectadas por Isolation Forest: {n_anomalies_iso} ({n_anomalies_iso/len(final_data)*100:.2f}%)")

In [ ]:
# Visualizar anomalías de Isolation Forest
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Temperatura vs Irradiancia
normal = final_data[final_data['anomaly_isolation'] == 1]
anomaly = final_data[final_data['anomaly_isolation'] == -1]

axes[0, 0].scatter(normal['ambient_temperature_o_149575'], normal['poa_irradiance_o_149574'], 
                   c='blue', alpha=0.5, label='Normal', s=10)
axes[0, 0].scatter(anomaly['ambient_temperature_o_149575'], anomaly['poa_irradiance_o_149574'], 
                   c='red', alpha=0.8, label='Anomalía', s=20)
axes[0, 0].set_xlabel('Temperatura Ambiente (°C)')
axes[0, 0].set_ylabel('Irradiancia POA (W/m²)')
axes[0, 0].set_title('Isolation Forest: Temperatura vs Irradiancia')
axes[0, 0].legend()

# Gráfico 2: Potencia Total vs Hora del día
axes[0, 1].scatter(normal['hour'], normal['total_ac_power'], 
                   c='blue', alpha=0.5, label='Normal', s=10)
axes[0, 1].scatter(anomaly['hour'], anomaly['total_ac_power'], 
                   c='red', alpha=0.8, label='Anomalía', s=20)
axes[0, 1].set_xlabel('Hora del día')
axes[0, 1].set_ylabel('Potencia AC Total (W)')
axes[0, 1].set_title('Isolation Forest: Potencia vs Hora')
axes[0, 1].legend()

# Gráfico 3: Distribución de scores de anomalía
axes[1, 0].hist(final_data['anomaly_score_isolation'], bins=50, edgecolor='black')
axes[1, 0].axvline(x=0, color='red', linestyle='--', label='Umbral')
axes[1, 0].set_xlabel('Score de Anomalía')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].set_title('Distribución de Scores - Isolation Forest')
axes[1, 0].legend()

# Gráfico 4: Serie temporal de anomalías
sample_data = final_data.iloc[:5000]  # Muestra para visualización
axes[1, 1].plot(sample_data['measured_on'], sample_data['total_ac_power'], 
                'b-', alpha=0.5, linewidth=0.5)
anomaly_sample = sample_data[sample_data['anomaly_isolation'] == -1]
axes[1, 1].scatter(anomaly_sample['measured_on'], anomaly_sample['total_ac_power'], 
                   c='red', s=20, label='Anomalías')
axes[1, 1].set_xlabel('Fecha')
axes[1, 1].set_ylabel('Potencia AC Total (W)')
axes[1, 1].set_title('Serie Temporal con Anomalías - Isolation Forest')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

### 4.2 Método 2: Mahalanobis Distance con Elliptic Envelope

In [ ]:
# Implementar Elliptic Envelope (basado en distancia de Mahalanobis)
elliptic = EllipticEnvelope(contamination=0.05, random_state=42)
final_data['anomaly_mahalanobis'] = elliptic.fit_predict(X_scaled)
final_data['mahalanobis_distance'] = elliptic.mahalanobis(X_scaled)

# Contar anomalías
n_anomalies_maha = (final_data['anomaly_mahalanobis'] == -1).sum()
print(f"Anomalías detectadas por Mahalanobis: {n_anomalies_maha} ({n_anomalies_maha/len(final_data)*100:.2f}%)")

In [ ]:
# Visualizar anomalías de Mahalanobis
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico 1: PCA para visualización 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

normal_maha = X_pca[final_data['anomaly_mahalanobis'] == 1]
anomaly_maha = X_pca[final_data['anomaly_mahalanobis'] == -1]

axes[0].scatter(normal_maha[:, 0], normal_maha[:, 1], c='blue', alpha=0.5, label='Normal', s=10)
axes[0].scatter(anomaly_maha[:, 0], anomaly_maha[:, 1], c='red', alpha=0.8, label='Anomalía', s=20)
axes[0].set_xlabel('Primera Componente Principal')
axes[0].set_ylabel('Segunda Componente Principal')
axes[0].set_title('Mahalanobis: Visualización PCA')
axes[0].legend()

# Gráfico 2: Distribución de distancias de Mahalanobis
axes[1].hist(final_data['mahalanobis_distance'], bins=50, edgecolor='black')
threshold = np.percentile(final_data['mahalanobis_distance'], 95)
axes[1].axvline(x=threshold, color='red', linestyle='--', label=f'Umbral 95% ({threshold:.2f})')
axes[1].set_xlabel('Distancia de Mahalanobis')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución de Distancias de Mahalanobis')
axes[1].legend()

plt.tight_layout()
plt.show()

### 4.3 Método 3: Local Outlier Factor (LOF)

In [ ]:
# Implementar Local Outlier Factor
# Nota: LOF requiere fit_predict en lugar de fit y predict separados
lof = LocalOutlierFactor(contamination=0.05, novelty=False, n_neighbors=20)
final_data['anomaly_lof'] = lof.fit_predict(X_scaled)
final_data['lof_score'] = lof.negative_outlier_factor_

# Contar anomalías
n_anomalies_lof = (final_data['anomaly_lof'] == -1).sum()
print(f"Anomalías detectadas por LOF: {n_anomalies_lof} ({n_anomalies_lof/len(final_data)*100:.2f}%)")

In [ ]:
# Visualizar comparación de los tres métodos
fig, ax = plt.subplots(figsize=(12, 8))

# Crear matriz de confusión entre métodos
methods = ['Isolation Forest', 'Mahalanobis', 'LOF']
confusion_matrix = np.zeros((3, 3))

# Calcular coincidencias
anomaly_cols = ['anomaly_isolation', 'anomaly_mahalanobis', 'anomaly_lof']
for i in range(3):
    for j in range(3):
        if i == j:
            confusion_matrix[i, j] = (final_data[anomaly_cols[i]] == -1).sum()
        else:
            confusion_matrix[i, j] = ((final_data[anomaly_cols[i]] == -1) & 
                                    (final_data[anomaly_cols[j]] == -1)).sum()

# Visualizar heatmap
sns.heatmap(confusion_matrix, annot=True, fmt='g', cmap='Blues', 
            xticklabels=methods, yticklabels=methods)
ax.set_title('Coincidencia de Anomalías entre Métodos')
ax.set_xlabel('Método')
ax.set_ylabel('Método')
plt.show()

## 5. Análisis de consenso y métricas finales

In [ ]:
# Crear score de consenso
final_data['anomaly_count'] = 0
for col in anomaly_cols:
    final_data['anomaly_count'] += (final_data[col] == -1).astype(int)

# Definir niveles de severidad
final_data['severity'] = 'Normal'
final_data.loc[final_data['anomaly_count'] == 1, 'severity'] = 'Bajo'
final_data.loc[final_data['anomaly_count'] == 2, 'severity'] = 'Medio'
final_data.loc[final_data['anomaly_count'] == 3, 'severity'] = 'Alto'

# Estadísticas de severidad
severity_counts = final_data['severity'].value_counts()
print("\n=== DISTRIBUCIÓN DE SEVERIDAD ===")
print(severity_counts)
print(f"\nPorcentaje de datos anómalos (al menos 1 método): {(final_data['anomaly_count'] > 0).sum() / len(final_data) * 100:.2f}%")
print(f"Porcentaje de consenso total (3 métodos): {(final_data['anomaly_count'] == 3).sum() / len(final_data) * 100:.2f}%")

In [ ]:
# Análisis de anomalías por inversor
# Verificar si hay inversores con más anomalías
inverter_anomalies = {}
for i in range(1, 25):  # 24 inversores
    inv_col = f'inv_{i:02d}_ac_power_inv_'
    inv_cols = [col for col in final_data.columns if inv_col in col]
    if inv_cols:
        # Calcular anomalías cuando este inversor tiene potencia baja pero otros no
        inv_power = final_data[inv_cols[0]]
        other_power = final_data['total_ac_power'] - inv_power
        
        # Detectar cuando este inversor está significativamente por debajo
        inv_anomaly = (inv_power < 0.7 * (final_data['total_ac_power'] / 24)) & \
                      (final_data['poa_irradiance_o_149574'] > 200)
        
        inverter_anomalies[f'Inversor_{i:02d}'] = inv_anomaly.sum()

# Mostrar inversores con más problemas
inv_df = pd.DataFrame.from_dict(inverter_anomalies, orient='index', columns=['Anomalías'])
inv_df = inv_df.sort_values('Anomalías', ascending=False)
print("\n=== TOP 10 INVERSORES CON MÁS ANOMALÍAS ===")
print(inv_df.head(10))

## 6. Exportar resultados y crear informe

In [ ]:
# Crear DataFrame con resumen de anomalías
anomaly_summary = final_data[final_data['anomaly_count'] > 0][[
    'measured_on', 'ambient_temperature_o_149575', 'poa_irradiance_o_149574',
    'total_ac_power', 'anomaly_count', 'severity'
]].copy()

# Agregar información sobre qué métodos detectaron la anomalía
anomaly_summary['detected_by'] = ''
for idx in anomaly_summary.index:
    methods_detected = []
    if final_data.loc[idx, 'anomaly_isolation'] == -1:
        methods_detected.append('Isolation')
    if final_data.loc[idx, 'anomaly_mahalanobis'] == -1:
        methods_detected.append('Mahalanobis')
    if final_data.loc[idx, 'anomaly_lof'] == -1:
        methods_detected.append('LOF')
    anomaly_summary.loc[idx, 'detected_by'] = ', '.join(methods_detected)

# Guardar resultados
anomaly_summary.to_csv('anomaly_report.csv', index=False)
print(f"\nReporte de anomalías guardado: {len(anomaly_summary)} anomalías detectadas")

# Mostrar muestra del reporte
print("\n=== MUESTRA DEL REPORTE DE ANOMALÍAS ===")
print(anomaly_summary.head(10))

## 7. Conclusiones y Recomendaciones

### Resumen de Hallazgos:

1. **Isolation Forest** detectó anomalías basándose en el aislamiento de puntos en el espacio de características
2. **Mahalanobis Distance** identificó puntos que se desvían significativamente de la distribución normal multivariada
3. **Local Outlier Factor** encontró anomalías basándose en la densidad local de los puntos

### Métrica de Severidad:
- **Normal**: No detectado por ningún método
- **Bajo**: Detectado por 1 método (posible falso positivo)
- **Medio**: Detectado por 2 métodos (anomalía probable)
- **Alto**: Detectado por 3 métodos (anomalía confirmada)

### Recomendaciones:
1. Investigar las anomalías de severidad "Alta" primero
2. Revisar los inversores con mayor número de anomalías
3. Correlacionar las anomalías con eventos de mantenimiento conocidos
4. Implementar alertas en tiempo real basadas en estos modelos